#**<font size="10" color='cornflowerblue'><ins>Welcome to exercise 8**
## **Note**: After choosing or changing an answer, run its block to save changes.
## **Note**: It is best to avoid Runtime -> Run all.



Block 1#


In [ ]:
#@title #**What is your ID and group number? (<font color="red">*Mandatory</font>)**


def initialize_colab():
  # Verify credentials
  if id_number < 1000 or group_number == 0 or group_number > 1000:
    raise Exception("Are you sure these details are correct?") 
  # Messages to user
  print("ID number saved: ", id_number)
  print("Group number saved: ", group_number)
  print("\nYou may now begin, good luck!")


try:
  # Imports and library initialization
  import warnings
  warnings.simplefilter(action='ignore', category=FutureWarning)
  import numpy as np
  import matplotlib.pyplot as plt
  import seaborn as sns
  import pandas as pd
  from math import sqrt
  from scipy import stats
  import statsmodels.api as sm
  from ipywidgets import interact
  sns.set(style = "white", palette = "muted", color_codes = True)
  sns.set(rc = {'figure.figsize': (10, 10)});
  sns.set_context("talk")

  # Setting up global colab variables
  exercise_number = 8
  executed_blocks = {0, 1}
  block_mapping = {2: 1, 3: 2, 4:3, 5:4, 6: 5.1, 7: 5.2, 8: 5.3, 9: 5.4,
                   10: 6.1, 11: 6.2, 12: 6.3, 14: 7.1, 15: 7.2, 16: 7.3,
                   17: 7.4, 19: 7.5, 20: 7.6, 21: 7.7, 22: 7.8, 23: 7.9, 
                   24: '7.10'}
  answers = {}
  data = {}
  id_number = 0 #@param {type:"integer"}
  group_number = 0 #@param {type:"integer"}
  initialize_colab()
  initialized = True
  
except Exception as e:
  print(str(e))


# Dependency check function
def verify_dependencies(block_id, required_blocks=[]):
  for block in required_blocks:
    if block not in executed_blocks:
      raise Exception("You must first run block "+str(block)+"#")



# **********************  Statistics helper functions  **********************
def t_fun(deg_freedom,deg_freedom_extra,t_score):
  pval = (stats.t.sf((t_score), deg_freedom))  # two-sided pvalue = Prob(abs(t)>tt)
  if (t_score<0):
    pval=1-pval #
  print ("t(",deg_freedom ,") = ", t_score,", p-value:", pval)
  ttt = stats.t.rvs(deg_freedom,size=5000)
  plot_dist(ttt, height=5,bins=20,color="orange", fit="t")
  plt.axvline(t_score,color="black",linestyle="--",label="Chosen value")
  plt.title("t-distribution")


def F_fun(df1,df2,F_score):
  df1 = float(df1) ##k-1
  df2 =float(df2) ##N-k
  F_score = float(F_score)  ##MSB/MSW
  pval = (stats.f.sf(F_score, df1,df2))  # two-sided pvalue = Prob(abs(t)>tt)
  print("F(",df1,",",df2 ,") = ", F_score,", p-value:", round(pval,3),"\n")
  fff = stats.f.rvs(df1,df2,size=1000)
  sns.distplot(fff,hist=1,kde=0,axlabel="F-score",fit=stats.f,fit_kws={"color":"purple"})
  plt.xlim(0,max(10,F_score*1.1))
  plt.axvline(F_score,color="black",linestyle="--")
  plt.title("F-distribution")


def plot_dist(dist, bins="auto", kde=False, fit="None", color="b", stat="density",height=8, aspect=1):
  plt.figure(0);
  sns.displot(dist, kde=kde,bins=bins,stat=stat,color=color,height=height, aspect=aspect);
  if (fit=="Normal") & (stat=="density"):  
    d = stats.norm.fit(dist)
    mu = d[0]; sigma = d[1]
    x = np.linspace(mu - 3.5*sigma, mu + 3.5*sigma, 200) #limits of dist
    plt.plot(x, stats.norm.pdf(x, mu, sigma),color="black");
  elif (fit=="F") & (stat=="density"):  
    d = stats.f.fit(dist)
    dfn = d[0]; dfd = d[1]
    interval = stats.f.interval(.995,dfn,dfd) #limits of dist
    x = np.linspace(interval[0],interval[1], 200)
    plt.plot(x, stats.f.pdf(x, dfn, dfd),color="black");
  elif (fit=="gamma") & (stat=="density"):  
    d = stats.gamma.fit(dist)
    a = d[0]; loc = d[1]; scale = d[2]
    interval = stats.gamma.interval(.995,a,loc,scale) #limits of dist
    x = np.linspace(interval[0],interval[1], 200)
    plt.plot(x, stats.gamma.pdf(x, a, loc, scale),color="black");
  elif (fit=="t") & (stat=="density"):  
    d = stats.t.fit(dist)
    a = d[0]; loc = d[1]; scale = d[2]
    interval = stats.t.interval(.995,a,loc,scale) #limits of dist
    x = np.linspace(interval[0],interval[1], 200)
    plt.plot(x, stats.t.pdf(x, a, loc, scale),color="black");


def perform_fdr_t(t_vals, deg_freedom, alpha):
  old_t=t_vals.copy()
  t_vals=np.sort(t_vals)[::-1]
  pvals = [(stats.t.sf(np.abs(t_score), deg_freedom)) for t_score in t_vals]
  sig=0
  for i in range(len(pvals)):
    if pvals[i]<(alpha*(i+1)/len(t_vals)):
      sig=pvals[i]
  return (sig)


def boot_dist(dist, B=10000,replacement = True):
  sample_matrix = np.random.choice(dist,size=(len(dist),B),replace=replacement)
  means_dist = np.mean(sample_matrix,axis=0) # average each column resulting in len(dist) X 1 vector
  return (means_dist)


def boot_dist_F(data,IV,DV, B=10000, replacement = True):
  levels = np.unique(data[IV])
  F_vals=[]
  for i in range(B):
    samples = [np.random.choice(data[data[IV]==level][DV],size=len(data[data[IV]==level][DV]),
                                replace=replacement) for level in levels]
    f,_=stats.f_oneway(*samples) # average each column resulting in len(dist) X 1 vector
    F_vals.append(f) 
  return (F_vals)


def cohen_d(x,y):
  nx = len(x)
  ny = len(y)
  dof = nx + ny - 2
  return (np.mean(x) - np.mean(y)) / np.sqrt(((nx-1)*np.std(x, ddof=1) ** 2 + (ny-1)*np.std(y, ddof=1) ** 2) / dof)


def r_score(x, y):
    return round(stats.pearsonr(x, y)[0],3)


def spearman_score(x, y):
    return round(stats.spearmanr(x, y)[0],3)

Block 2#

In [ ]:
#@title **<font size="6" color='cornflowerblue'><ins>Question 1</ins>** 
#@markdown #Below is a scatter plot of two variables. A correlation was calculated for the variables, and then an outlier was added to the sample. The outlier is marked with an arrow.
#@markdown <img src="https://drive.google.com/uc?export=view&id=1MRqpFaaHohjJBbmUNwka7bmF0WErg64A"></img>
#@markdown #**Which of the following is necessarily true after adding the outlier observation?**

block_id = 2
required_blocks = []

def prepare_block():
  try:
    initialized
  except:
    raise Exception("You need to identify yourself in Block 1#")
  verify_dependencies(block_id, required_blocks)


def block2():
  Answer = 'Choose the correct answer' #@param ["Choose the correct answer", "Both the Pearson correlation and the Spearman correlation will grow, but the Spearman correlation will grow more.", "Both the Pearson correlation and the Spearman correlation will grow, but the Pearson correlation will grow more.", "Both the Pearson correlation and the Spearman correlation will get smaller, but the Spearman correlation will be more affected.", "Both the Pearson correlation and the Spearman correlation will get smaller, but the Pearson correlation will be more affected."]
  if Answer == 'Choose the correct answer':
    raise Exception("You must choose an answer")
  answers[block_id] = Answer
  print("Your Answer is: ", answers[block_id])


try:
  prepare_block()  
  eval('block'+str(block_id))()
  executed_blocks.add(block_id)
except Exception as e:
  print(str(e))


Block 3#

In [ ]:
#@title **<font size="6" color='cornflowerblue'><ins>Question 2</ins>** 
#@markdown #**The researcher Almaz wanted to test the correlation between two variables. If she is interested in the degree of the monotony of the relationship, she should calculate a ________ correlation. If she is interested in the degree of linearity of the relationship, she should calculate _____.**

block_id = 3
required_blocks = []

def prepare_block():
  try:
    initialized
  except:
    raise Exception("You need to identify yourself in Block 1#")
  verify_dependencies(block_id, required_blocks)


def block3():
  Answer = 'Choose the correct answer' #@param ["Choose the correct answer", "Pearson, Linear Regression", "Spearman, Pearson", "Pearson, Spearman", "Spearman, Two-way ANOVA"]
  if Answer == 'Choose the correct answer':
    raise Exception("You must choose an answer")
  answers[block_id] = Answer
  print("Your Answer is: ", answers[block_id])
  return None

try:
  prepare_block()
  eval('block'+str(block_id))()
  executed_blocks.add(block_id)
except Exception as e:
  print(str(e))


Block 4#

In [ ]:
#@title **<font size="6" color='cornflowerblue'><ins>Question 3</ins>** 
#@markdown #**When ________ correlation equals 1, ________ correlation is necessarily equal to _____________.**

block_id = 4
required_blocks = []

def prepare_block():
  try:
    initialized
  except:
    raise Exception("You need to identify yourself in Block 1#")
  verify_dependencies(block_id, required_blocks)


def block4():
  Answer = 'Choose the correct answer' #@param ["Choose the correct answer", "Spearman, Pearson, 1", "Pearson, Spearman, 1", "Spearman, Pearson, -1", "Pearson, Spearman, -1"]
  if Answer == 'Choose the correct answer':
    raise Exception("You must choose an answer")
  answers[block_id] = Answer
  print("Your Answer is: ", answers[block_id])


try:
  prepare_block()
  eval('block'+str(block_id))()
  executed_blocks.add(block_id)
except Exception as e:
  print(str(e))


Block 5#

In [ ]:
#@title **<font size="6" color='cornflowerblue'><ins>Question 4</ins>** 
#@markdown #**Which of the following describes a negative correlation?**

block_id = 5
required_blocks = []

def prepare_block():
  try:
    initialized
  except:
    raise Exception("You need to identify yourself in Block 1#")
  verify_dependencies(block_id, required_blocks)


def block5():
  Answer = 'Choose the correct answer' #@param ["Choose the correct answer", "The more people get vaccinated, the fewer flu cases there are.", "The less rain there is, the less green the grass is", "The more chocolate we eat, the happier we are.", "The less episodes of the ''Grey's Anatomy'' we watch, the smaller is our emotional range."]
  if Answer == 'Choose the correct answer':
    raise Exception("You must choose an answer")
  answers[block_id] = Answer
  print("Your Answer is: ", answers[block_id])


try:
  prepare_block()
  eval('block'+str(block_id))()
  executed_blocks.add(block_id)
except Exception as e:
  print(str(e))


Block 6#

In [ ]:
#@title **<font size="6" color='cornflowerblue'><ins>Question 5.1</ins>** 
#@markdown #**Mark which correlation test will better represent the data**
#@markdown <img src="https://drive.google.com/uc?export=view&id=1F2WdYyZspcmYrOgDHeCGpUUO0r1I1880"></img>

block_id = 6
required_blocks = []

def prepare_block():
  try:
    initialized
  except:
    raise Exception("You need to identify yourself in Block 1#")
  verify_dependencies(block_id, required_blocks)


def block6():
  Answer = 'Choose the correct answer' #@param ["Choose the correct answer", "Pearson correlation", "Spearman correlation", "None of them"]
  if Answer == 'Choose the correct answer':
    raise Exception("You must choose an answer")
  answers[block_id] = Answer
  print("Your Answer is: ", answers[block_id])


try:
  prepare_block()
  eval('block'+str(block_id))()
  executed_blocks.add(block_id)
except Exception as e:
  print(str(e))


Block 7#

In [ ]:
#@title **<font size="6" color='cornflowerblue'><ins>Question 5.2</ins>** 
#@markdown #**Mark which correlation test will show a higher correlation**
#@markdown <img src="https://drive.google.com/uc?export=view&id=1i37ajcWf6LP_vJmE162rwb2IkGEczX4G"></img>

block_id = 7
required_blocks = []

def prepare_block():
  try:
    initialized
  except:
    raise Exception("You need to identify yourself in Block 1#")
  verify_dependencies(block_id, required_blocks)


def block7():
  Answer = 'Choose the correct answer' #@param ["Choose the correct answer", "Pearson correlation", "Spearman correlation", "None of them"]
  if Answer == 'Choose the correct answer':
    raise Exception("You must choose an answer")
  answers[block_id] = Answer
  print("Your Answer is: ", answers[block_id])


try:
  prepare_block()
  eval('block'+str(block_id))()
  executed_blocks.add(block_id)
except Exception as e:
  print(str(e))


Block 8#

In [ ]:
#@title **<font size="6" color='cornflowerblue'><ins>Question 5.3</ins>** 
#@markdown #**Mark which correlation test will better represent the data**
#@markdown <img src="https://drive.google.com/uc?export=view&id=1y0j6GiTJlarZalU2iuQdGd-z0Q6YM6NI"></img>

block_id = 8
required_blocks = []

def prepare_block():
  try:
    initialized
  except:
    raise Exception("You need to identify yourself in Block 1#")
  verify_dependencies(block_id, required_blocks)


def block8():
  Answer = 'Choose the correct answer' #@param ["Choose the correct answer", "Pearson correlation", "Spearman correlation", "None of them"]
  if Answer == 'Choose the correct answer':
    raise Exception("You must choose an answer")
  answers[block_id] = Answer
  print("Your Answer is: ", answers[block_id])


try:
  prepare_block()
  eval('block'+str(block_id))()
  executed_blocks.add(block_id)
except Exception as e:
  print(str(e))


Block 9#

In [ ]:
#@title **<font size="6" color='cornflowerblue'><ins>Question 5.4</ins>** 
#@markdown #**Mark which correlation test will better represent the data**
#@markdown <img src="https://drive.google.com/uc?export=view&id=1-21bcESMoKnJPjB8hv3z5VLqRz6ERgZl"></img>

block_id = 9
required_blocks = []

def prepare_block():
  try:
    initialized
  except:
    raise Exception("You need to identify yourself in Block 1#")
  verify_dependencies(block_id, required_blocks)


def block9():
  Answer = 'Choose the correct answer' #@param ["Choose the correct answer", "Pearson correlation", "Spearman correlation", "None of them"]
  if Answer == 'Choose the correct answer':
    raise Exception("You must choose an answer")
  answers[block_id] = Answer
  print("Your Answer is: ", answers[block_id])


try:
  prepare_block()
  eval('block'+str(block_id))()
  executed_blocks.add(block_id)
except Exception as e:
  print(str(e))


#**<font size="6" color='cornflowerblue'><ins>Question 6</ins>** 
#You have the ability to control one point in the data below (run to see data). Your task is to select the minimal distance of the point from the center and create an outlier that causes the correlation to be significantly larger than zero with p < 0.05, p < 0.01 and p < 0.001.

Hint: run the question a few times and see what happens when you put the values in the p-value calculator

Block 10#

In [ ]:
#@title **<font size="6" color='cornflowerblue'><ins>Question 6.1</ins>** 
#@markdown #**Set the minimum distance from center that creates a positive correlation that is significant at p < 0.05**

block_id = 10
required_blocks = []

def prepare_block():
  try:
    initialized
  except:
    raise Exception("You need to identify yourself in Block 1#")
  verify_dependencies(block_id, required_blocks)


def block10():
  distance_from_center_1 = -20 #@param {type:"slider", min:-20, max:20, step:0.25}
  n=40

  ##empty circle
  X = np.linspace(-4, 4, n)
  np.random.seed(group_number)
  Y = (np.random.choice([-1,1],n))*np.sqrt(16.1-X**2)+np.random.normal(0,.1,n)

  ##full circle
  rho = np.sqrt(np.random.uniform(0, 1, n))
  phi = np.random.uniform(0, 2*np.pi, n)
  X = 2.5*rho * np.cos(phi)
  Y = 2.5*rho * np.sin(phi)
  X[1]=distance_from_center_1
  Y[1]=distance_from_center_1

  Curr_data = pd.DataFrame({"X":X,"Y":Y})
  sns.regplot(x="X", y="Y" ,data=Curr_data, color = "darkred",fit_reg=False,ci=None,scatter_kws={"s": 40})
  pears1=stats.pearsonr(X,Y)[1]/2
  plt.title("Pearson correlation, r = "+str(r_score(X,Y))+ "\nN = "+str(len(X)));
  plt.xlabel("x");
  plt.ylabel("y");
  answers[block_id] = distance_from_center_1
  data[block_id] = {'pears': pears1, 'dist': distance_from_center_1}
  print("Your answer:", answers[block_id])


try:
  prepare_block()
  eval('block'+str(block_id))()
  executed_blocks.add(block_id)
except Exception as e:
  print(str(e))

Block 11#

In [ ]:
#@title **<font size="6" color='cornflowerblue'><ins>Question 6.2</ins>** 
#@markdown #**Set the minimum distance from center that creates a positive correlation that is significant at p < 0.01**

block_id = 11
required_blocks = []

def prepare_block():
  try:
    initialized
  except:
    raise Exception("You need to identify yourself in Block 1#")
  verify_dependencies(block_id, required_blocks)


def block11():
  distance_from_center_2=-20 #@param {type:"slider", min:-20, max:20, step:0.25}
  n=40
  ##empty circle
  X = np.linspace(-4, 4, n)
  np.random.seed(group_number)
  Y = (np.random.choice([-1,1],n))*np.sqrt(16.1-X**2)+np.random.normal(0,.1,n)

  ##full circle
  rho = np.sqrt(np.random.uniform(0, 1, 40))
  phi = np.random.uniform(0, 2*np.pi, 40)
  X = 2.5*rho * np.cos(phi)
  Y = 2.5*rho * np.sin(phi)
  X[1]=distance_from_center_2
  Y[1]=distance_from_center_2

  Curr_data = pd.DataFrame({"X":X,"Y":Y})
  sns.regplot(x="X", y="Y" ,data=Curr_data, color = "darkred",fit_reg=False,ci=None,scatter_kws={"s": 40})
  pears2=stats.pearsonr(X,Y)[1]/2
  plt.title("Pearson correlation, r = "+str(r_score(X,Y))+ "\nN = "+str(len(X)));
  plt.xlabel("x");
  plt.ylabel("y");

  answers[block_id] = distance_from_center_2
  data[block_id] = {'pears': pears2, 'dist': distance_from_center_2}
  print("Your answer:", answers[block_id])


try:
  prepare_block()
  eval('block'+str(block_id))()
  executed_blocks.add(block_id)
except Exception as e:
  print(str(e))

Block 12#

In [ ]:
#@title **<font size="6" color='cornflowerblue'><ins>Question 6.3</ins>** 
#@markdown #**Set the minimum distance from center that creates a positive correlation that is significant at the p < 0.001**

block_id = 12
required_blocks = []

def prepare_block():
  try:
    initialized
  except:
    raise Exception("You need to identify yourself in Block 1#")
  verify_dependencies(block_id, required_blocks)


def block12():
  distance_from_center_3=-20 #@param {type:"slider", min:-20, max:20, step:0.25}
  n=40

  ##empty circle
  X = np.linspace(-4, 4, n)
  np.random.seed(group_number)
  Y = (np.random.choice([-1,1],n))*np.sqrt(16.1-X**2)+np.random.normal(0,.1,n)

  ##full circle
  rho = np.sqrt(np.random.uniform(0, 1, n))
  phi = np.random.uniform(0, 2*np.pi, n)
  X = 2.5*rho * np.cos(phi)
  Y = 2.5*rho * np.sin(phi)
  X[1]=distance_from_center_3
  Y[1]=distance_from_center_3

  Curr_data = pd.DataFrame({"X":X,"Y":Y})
  sns.regplot(x="X", y="Y" ,data=Curr_data, color = "darkred",fit_reg=False,ci=None,scatter_kws={"s": 40})
  pears3=stats.pearsonr(X,Y)[1]/2
  plt.title("Pearson correlation, r = "+str(r_score(X,Y))+ "\nN = "+str(len(X)));
  plt.xlabel("x");
  plt.ylabel("y");

  answers[block_id] = distance_from_center_3
  data[block_id] = {'pears': pears3, 'dist': distance_from_center_3}
  print("Your answer:", answers[block_id])


try:
  prepare_block()
  eval('block'+str(block_id))()
  executed_blocks.add(block_id)
except Exception as e:
  print(str(e))

Block 13#

In [ ]:
#@title **<font size="6" color='cornflowerblue'><ins>Question 7</ins>** 
#@markdown #In this question you will produce the data-driven estimations for the  H0  distribution of the correlation and the "real world" correlation distribution and learn how to draw the right conclusions from it.
#@markdown #**The next 4 questions concern a new dataset dataset (run to see data and correlation)**

block_id = 13
required_blocks = []


B=10000
def samp_replacement(x,y,a=None,b=None,c=None):
  samples = np.random.choice(np.arange(len(x)),len(x),replace=True)
  return (x[samples],y[samples])

def samp_replacement_x_only(x,y,a=None,b=None,c=None):
  samples = np.random.choice(np.arange(len(x)),len(x),replace=True)
  return (x[samples],y)

def shuffle_values(x,y,a=None,b=None,c=None):
  samples_x = np.random.choice(np.arange(len(x)),len(x),replace=False)
  samples_y = np.random.choice(np.arange(len(x)),len(x),replace=False)
  return (x[samples_x],y[samples_y])

def calculate_correlation(x,y,a=None,b=None,c=None):
  return (stats.pearsonr(x,y)[0])

def compare_means(x,y):
  return np.mean(x-y)

def repeat_previous_two(fun1,fun2,x,y,B=B,c=None):
  correl_vec = []
  np.random.seed(1) #should be 1 not group number
  for i in range(B):
    pairs=fun1(x,y)
    curr_cor=fun2(pairs[0],pairs[1])
    correl_vec.append(curr_cor)
  return (correl_vec)

def create_dist(fun1,fun2,fun3,x,y):
  correlations = fun3(fun1,fun2,x,y) 
  plt.figure(figsize=(7,7)) 
  sns.histplot(correlations, bins=30,color="orange",stat="density")
  sns.kdeplot(correlations,color="black")


def prepare_block():
  try:
    initialized
  except:
    raise Exception("You need to identify yourself in Block 1#")
  verify_dependencies(block_id, required_blocks)


def block13():
  np.random.seed(int(group_number+1))
  n=np.random.choice(np.arange(25,31))
  x=np.random.normal(70,10,n)

  correl=0.4

  flip=np.random.choice([-1,1])
  w1=correl*(.735);w2 = 1-w1 #create correlation with equal variances

  y=flip*(w1*x+w2*np.random.normal(70,10,n))

  pval_71 =np.round(stats.pearsonr(x,y)[1],3)
  cor_71 =np.round(stats.pearsonr(x,y)[0],3)
  print(f"x: {x}\n")
  print(f"y: {y}\n")
  print(f"N={n}, \033[1mpearson correlation={cor_71}, p={pval_71}")

  step_dict = {"Sample with replacement from the X values.":samp_replacement_x_only,
              "Compare the mean of the X scores with the mean of the Y scores":compare_means,
              "Sample with replacement from the pairs":samp_replacement,
              "Calculate the correlation (Pearson or Spearman).":calculate_correlation,
              "Repeat the previous steps 10,000 times.":repeat_previous_two,
              "Mix the Y values between the different X values.":shuffle_values,
              "Create a distribution of all the correlation coefficients that were calculated.":create_dist}

  return n, x, y, cor_71, pval_71, step_dict

try:
  prepare_block()
  n, x, y, cor_71, pval_71, step_dict = eval('block'+str(block_id))()
  executed_blocks.add(block_id)
except Exception as e:
  print(str(e))


Block 14#

In [ ]:
#@title **<font size="6" color='cornflowerblue'><ins>Question 7.1</ins>** 
#@markdown #**Choose the steps to produce the H0 distribution of the correlation and order them by the right order (not all of them must be used - fill "X" in excessive steps).** 
#@markdown #**There is only one correct answer.**

block_id = 14
required_blocks = [13]

def prepare_block():
  try:
    initialized
  except:
    raise Exception("You need to identify yourself in Block 1#")
  verify_dependencies(block_id, required_blocks)


try:
  prepare_block()
  step_1a = "X"#@param ["X","Mix the Y values between the different X values.","Sample with replacement from the pairs","Create a distribution of all the correlation coefficients that were calculated.","Sample with replacement from the X values.","Choose a new threshold for rejecting H0 (alpha)","Compare the mean of the X scores with the mean of the Y scores","Calculate the correlation (Pearson or Spearman).","Repeat the previous steps 10,000 times."]
  if step_1a == 'X':
    raise Exception("You must choose step_1.")
  step_2a = "X"#@param ["X","Mix the Y values between the different X values.","Sample with replacement from the pairs","Create a distribution of all the correlation coefficients that were calculated.","Sample with replacement from the X values.","Choose a new threshold for rejecting H0 (alpha)","Compare the mean of the X scores with the mean of the Y scores","Calculate the correlation (Pearson or Spearman).","Repeat the previous steps 10,000 times."]
  step_3a = "X"#@param ["X","Mix the Y values between the different X values.","Sample with replacement from the pairs","Create a distribution of all the correlation coefficients that were calculated.","Sample with replacement from the X values.","Choose a new threshold for rejecting H0 (alpha)","Compare the mean of the X scores with the mean of the Y scores","Calculate the correlation (Pearson or Spearman).","Repeat the previous steps 10,000 times."]
  step_4a = "X"#@param ["X","Mix the Y values between the different X values.","Sample with replacement from the pairs","Create a distribution of all the correlation coefficients that were calculated.","Sample with replacement from the X values.","Choose a new threshold for rejecting H0 (alpha)","Compare the mean of the X scores with the mean of the Y scores","Calculate the correlation (Pearson or Spearman).","Repeat the previous steps 10,000 times."]
  step_5a = "X"#@param ["X","Mix the Y values between the different X values.","Sample with replacement from the pairs","Create a distribution of all the correlation coefficients that were calculated.","Sample with replacement from the X values.","Choose a new threshold for rejecting H0 (alpha)","Compare the mean of the X scores with the mean of the Y scores","Calculate the correlation (Pearson or Spearman).","Repeat the previous steps 10,000 times."]
  step_6a = "X"#@param ["X","Mix the Y values between the different X values.","Sample with replacement from the pairs","Create a distribution of all the correlation coefficients that were calculated.","Sample with replacement from the X values.","Choose a new threshold for rejecting H0 (alpha)","Compare the mean of the X scores with the mean of the Y scores","Calculate the correlation (Pearson or Spearman).","Repeat the previous steps 10,000 times."]
  step_7a = "X"#@param ["X","Mix the Y values between the different X values.","Sample with replacement from the pairs","Create a distribution of all the correlation coefficients that were calculated.","Sample with replacement from the X values.","Choose a new threshold for rejecting H0 (alpha)","Compare the mean of the X scores with the mean of the Y scores","Calculate the correlation (Pearson or Spearman).","Repeat the previous steps 10,000 times."]
  # Processing answers
  answers[block_id] = (step_1a, step_2a, step_3a, step_4a, step_5a, step_6a, step_7a)
  print("Your answer:", answers[block_id])
  executed_blocks.add(block_id)
except Exception as e:
  print(str(e))

Block 15#


In [ ]:
#@title **<font size="6" color='cornflowerblue'><ins>Question 7.2</ins>** 
#@markdown #**How would you visualize the results to estimate the H0 distribution?**
#@markdown - **Note** - A good graph here doesn't guarentee that you were right in the previous question

block_id = 15
required_blocks = [13]

def prepare_block():
  try:
    initialized
  except:
    raise Exception("You need to identify yourself in Block 1#")
  verify_dependencies(block_id, required_blocks)


def block15():
  x_axis = "Choose an answer" #@param ["Choose an answer","observation number","x values from the graph above", "calculated correlations"]
  y_axis = "Choose an answer" #@param ["Choose an answer","probability density", "y values from the graph above", "calculated correlations"]

  #plot their steps
  step_dict = {"Sample with replacement from the X values.":samp_replacement_x_only,
              "Compare the mean of the X scores with the mean of the Y scores":compare_means,
              "Sample with replacement from the pairs":samp_replacement,
              "Calculate the correlation (Pearson or Spearman).":calculate_correlation,
              "Repeat the previous steps 10,000 times.":repeat_previous_two,
              "Mix the Y values between the different X values.":shuffle_values,
              "Create a distribution of all the correlation coefficients that were calculated.":create_dist}
  steps = [step_dict[step_1a],step_dict[step_2a],step_dict[step_3a],step_dict[step_4a]]#by their choice

  if (x_axis=="x values from the graph above") and (y_axis=="probability density"):
    plt.hist(x)
  if (x_axis=="x values from the graph above") and (y_axis=="calculated correlations"):
    print("Cannot create this graph! choose x and y with the same number of observations")
  if (x_axis=="x values from the graph above") and (y_axis=="y values from the graph above"):
    plt.figure(figsize=(8,6));
    sns.regplot(x=x, y=y ,fit_reg=False, color = "orange")
    plt.title(f"N={n}, pearson correlation={cor_71}, p={pval_71}");

  if x_axis == "observation number":
    if y_axis=="calculated correlations":
          print("Cannot create this graph! choose x and y with the same number of observations")
    elif y_axis == "probability density":
      plt.hist(np.arange(n))
    else:
      plt.plot(np.arange(n),y)


  if (x_axis == "calculated correlations"):
    if (y_axis == "calculated correlations"):
      calculated=step_dict[step_3a](step_dict[step_1a],step_dict[step_2a],x,y)
      print(9)
      plt.plot(calculated,calculated)
    elif (y_axis == "y values from the graph above"):
      print("Cannot create this graph! choose x and y with the same number of observations")
    elif (y_axis == "probability density"):
      try:
        steps[3](step_dict[step_1a],step_dict[step_2a],step_dict[step_3a],x,y) 
      except:
        if step_4a ==  "Create a distribution of all the correlation coefficients that were calculated.":
          if step_3a == "Repeat the previous two steps 10,000 times.":
            plt.figure(figsize=(7,7)) 
            plt.hist(np.random.normal(loc=.7,scale=.1,size=10000),bins=30, color="orange")
          else:
            plt.figure(figsize=(7,7)) 
            plt.hist(np.repeat(cor_71,10000),bins=30, color="orange")
        if step_4a !=  "Create a distribution of all the correlation coefficients that were calculated.":
          print("You didn't plot anything.")
  plt.title("Your distribution");
  answers[block_id] = (x_axis, y_axis)
  print("Your answer:", answers[block_id])


try:
  prepare_block()
  eval('block'+str(block_id))()
  executed_blocks.add(block_id)
except Exception as e:
  print(str(e))

Block 16#

In [ ]:
#@title **<font size="6" color='cornflowerblue'><ins>Question 7.3</ins>** 
#@markdown #**Choose the steps to produce the correlation distribution of the real data and order them by the right order (not all of them must be used - fill "X" in excessive steps).** 
#@markdown #**There is only one correct answer.**

block_id = 16
required_blocks = [13]

def prepare_block():
  try:
    initialized
  except:
    raise Exception("You need to identify yourself in Block 1#")
  verify_dependencies(block_id, required_blocks)


try:
  prepare_block()
  step_1b = "X"#@param ["X","Mix the Y values between the different X values.","Sample with replacement from the pairs","Create a distribution of all the correlation coefficients that were calculated.","Sample with replacement from the X values.","Choose a new threshold for rejecting H0 (alpha)","Compare the mean of the X scores with the mean of the Y scores","Calculate the correlation (Pearson or Spearman).","Repeat the previous steps 10,000 times."]
  if step_1b == 'X':
    raise Exception("You must choose step_1.")
  step_2b = "X"#@param ["X","Mix the Y values between the different X values.","Sample with replacement from the pairs","Create a distribution of all the correlation coefficients that were calculated.","Sample with replacement from the X values.","Choose a new threshold for rejecting H0 (alpha)","Compare the mean of the X scores with the mean of the Y scores","Calculate the correlation (Pearson or Spearman).","Repeat the previous steps 10,000 times."]
  step_3b = "X"#@param ["X","Mix the Y values between the different X values.","Sample with replacement from the pairs","Create a distribution of all the correlation coefficients that were calculated.","Sample with replacement from the X values.","Choose a new threshold for rejecting H0 (alpha)","Compare the mean of the X scores with the mean of the Y scores","Calculate the correlation (Pearson or Spearman).","Repeat the previous steps 10,000 times."]
  step_4b = "X"#@param ["X","Mix the Y values between the different X values.","Sample with replacement from the pairs","Create a distribution of all the correlation coefficients that were calculated.","Sample with replacement from the X values.","Choose a new threshold for rejecting H0 (alpha)","Compare the mean of the X scores with the mean of the Y scores","Calculate the correlation (Pearson or Spearman).","Repeat the previous steps 10,000 times."]
  step_5b = "X"#@param ["X","Mix the Y values between the different X values.","Sample with replacement from the pairs","Create a distribution of all the correlation coefficients that were calculated.","Sample with replacement from the X values.","Choose a new threshold for rejecting H0 (alpha)","Compare the mean of the X scores with the mean of the Y scores","Calculate the correlation (Pearson or Spearman).","Repeat the previous steps 10,000 times."]
  step_6b = "X"#@param ["X","Mix the Y values between the different X values.","Sample with replacement from the pairs","Create a distribution of all the correlation coefficients that were calculated.","Sample with replacement from the X values.","Choose a new threshold for rejecting H0 (alpha)","Compare the mean of the X scores with the mean of the Y scores","Calculate the correlation (Pearson or Spearman).","Repeat the previous steps 10,000 times."]
  step_7b = "X"#@param ["X","Mix the Y values between the different X values.","Sample with replacement from the pairs","Create a distribution of all the correlation coefficients that were calculated.","Sample with replacement from the X values.","Choose a new threshold for rejecting H0 (alpha)","Compare the mean of the X scores with the mean of the Y scores","Calculate the correlation (Pearson or Spearman).","Repeat the previous steps 10,000 times."]
  # Processing answers
  answers[block_id] = (step_1b, step_2b, step_3b, step_4b, step_5b, step_6b, step_7b)
  print("Your answer:", answers[block_id])
  executed_blocks.add(block_id)
except Exception as e:
  print(str(e))

Block 17#

In [ ]:
#@title **<font size="6" color='cornflowerblue'><ins>Question 7.4</ins>** 
#@markdown #**How would you visualize this estimation of the sampling distribution of the correlation?**
#@markdown **Note** - A good graph here doesn't 100% guarentee that you were right in the previous question.

block_id = 17
required_blocks = [13]

def prepare_block():
  try:
    initialized
  except:
    raise Exception("You need to identify yourself in Block 1#")
  verify_dependencies(block_id, required_blocks)


def block17():
  x_axis = "Choose an answer" #@param ["Choose an answer","observation number","x values from the graph above", "calculated correlations"]
  y_axis = "Choose an answer" #@param ["Choose an answer","probability density", "y values from the graph above", "calculated correlations"]

  #plot their steps
  B=10000
  steps = [step_dict[step_1b],step_dict[step_2b],step_dict[step_3b],step_dict[step_4b]]#by their choice
  if (x_axis=="x values from the graph above") and (y_axis=="probability density"):
    plt.hist(x)
  if (x_axis=="x values from the graph above") and (y_axis=="calculated correlations"):
    print("Cannot create this graph! choose x and y with the same number of observations")
  if (x_axis=="x values from the graph above") and (y_axis=="y values from the graph above"):
    plt.figure(figsize=(8,6));
    sns.regplot(x=x, y=y ,fit_reg=False, color = "orange")
    plt.title(f"N={n}, pearson correlation={cor_71}, p={pval_71}");
  if (x_axis == "calculated correlations"):
    if (y_axis == "calculated correlations"):
      calculated=step_dict[step_3b](step_dict[step_1b],step_dict[step_2b],x,y)
      print(9)
      plt.plot(calculated,calculated)
    elif (y_axis == "y values from the graph above"):
      print("Cannot create this graph! choose x and y with the same number of observations")
    elif (y_axis == "probability density"):
      try:
        steps[3](step_dict[step_1b],step_dict[step_2b],step_dict[step_3b],x,y)
      except:
        if step_4b ==  "Create a distribution of all the correlation coefficients that were calculated.":
          if step_3b == "Repeat the previous two steps 10,000 times.":
            plt.figure(figsize=(7,7)) 
            plt.hist(np.random.normal(loc=.7,scale=.1,size=10000),bins=30, color="darkgreen")
          else:
            plt.figure(figsize=(7,7)) 
            plt.hist(np.repeat(cor_71,10000),bins=30, color="orange")
        if step_4b !=  "Create a distribution of all the correlation coefficients that were calculated.":
          print("You didn't plot anything.")
  if x_axis == "observation number":
    if y_axis=="calculated correlations":
          print("Cannot create this graph! choose x and y with the same number of observations")
    elif y_axis == "probability density":
      plt.hist(np.arange(n))
    else:
      plt.plot(np.arange(n),y)
  plt.title("Your distribution");

  answers[block_id] = (x_axis, y_axis)
  print("Your Answer is: ", answers[block_id])


try:
  prepare_block()
  eval('block'+str(block_id))()
  executed_blocks.add(block_id)
except Exception as e:
  print(str(e))


Block 18#

In [ ]:
#@title **<font size="6" color='cornflowerblue'><ins>Dataset 2#</ins>** 
#@markdown #The next 4 questions concern a new dataset.
#@markdown #**Run to see data and correlation**

block_id = 18
required_blocks = [13]

def prepare_block():
  try:
    initialized
  except:
    raise Exception("You need to identify yourself in Block 1#")
  verify_dependencies(block_id, required_blocks)


def block18():
  np.random.seed(int(group_number))
  n5=np.random.choice(np.arange(25,31))
  x5=np.random.normal(70,10,n5)
  correl = 0.4
  w1=correl*(.735);w2 = 1-w1 #create correlation with equal variances
  y5=(np.random.normal(70,10,n5))
  y5[6] = 120.937454
  x5[6] = 140.232782
  pval_75 =np.round(stats.pearsonr(x5,y5)[1],3)
  cor_75 =np.round(stats.pearsonr(x5,y5)[0],3)
  print(f"x: {x5}\n")
  print(f"y: {y5}\n")
  print(f"N={n5}, \033[1mpearson correlation={cor_75}, p={pval_75}")
  return n5, x5, y5, cor_75, pval_75


try:
  prepare_block()
  n5, x5, y5, cor_75, pval_75 = eval('block'+str(block_id))()
  executed_blocks.add(block_id)
except Exception as e:
  print(str(e))

Block 19#

In [ ]:
#@title **<font size="6" color='cornflowerblue'><ins>Question 7.5</ins>** 
#@markdown #**Choose the steps to produce the H0 distribution of the correlation and order them by the right order (not all of them must be used - fill "X" in excessive steps).** 
#@markdown #**There is only one correct answer.**

block_id = 19
required_blocks = [13, 18]

def prepare_block():
  try:
    initialized
  except:
    raise Exception("You need to identify yourself in Block 1#")
  verify_dependencies(block_id, required_blocks)


try:
  prepare_block()
  step_1c = "X"#@param ["X","Mix the Y values between the different X values.","Sample with replacement from the pairs","Create a distribution of all the correlation coefficients that were calculated.","Sample with replacement from the X values.","Choose a new threshold for rejecting H0 (alpha)","Compare the mean of the X scores with the mean of the Y scores","Calculate the correlation (Pearson or Spearman).","Repeat the previous steps 10,000 times."]
  if step_1c == 'X':
    raise Exception("You must choose step_1.")
  step_2c = "X"#@param ["X","Mix the Y values between the different X values.","Sample with replacement from the pairs","Create a distribution of all the correlation coefficients that were calculated.","Sample with replacement from the X values.","Choose a new threshold for rejecting H0 (alpha)","Compare the mean of the X scores with the mean of the Y scores","Calculate the correlation (Pearson or Spearman).","Repeat the previous steps 10,000 times."]
  step_3c = "X"#@param ["X","Mix the Y values between the different X values.","Sample with replacement from the pairs","Create a distribution of all the correlation coefficients that were calculated.","Sample with replacement from the X values.","Choose a new threshold for rejecting H0 (alpha)","Compare the mean of the X scores with the mean of the Y scores","Calculate the correlation (Pearson or Spearman).","Repeat the previous steps 10,000 times."]
  step_4c = "X"#@param ["X","Mix the Y values between the different X values.","Sample with replacement from the pairs","Create a distribution of all the correlation coefficients that were calculated.","Sample with replacement from the X values.","Choose a new threshold for rejecting H0 (alpha)","Compare the mean of the X scores with the mean of the Y scores","Calculate the correlation (Pearson or Spearman).","Repeat the previous steps 10,000 times."]
  step_5c = "X"#@param ["X","Mix the Y values between the different X values.","Sample with replacement from the pairs","Create a distribution of all the correlation coefficients that were calculated.","Sample with replacement from the X values.","Choose a new threshold for rejecting H0 (alpha)","Compare the mean of the X scores with the mean of the Y scores","Calculate the correlation (Pearson or Spearman).","Repeat the previous steps 10,000 times."]
  step_6c = "X"#@param ["X","Mix the Y values between the different X values.","Sample with replacement from the pairs","Create a distribution of all the correlation coefficients that were calculated.","Sample with replacement from the X values.","Choose a new threshold for rejecting H0 (alpha)","Compare the mean of the X scores with the mean of the Y scores","Calculate the correlation (Pearson or Spearman).","Repeat the previous steps 10,000 times."]
  step_7c = "X"#@param ["X","Mix the Y values between the different X values.","Sample with replacement from the pairs","Create a distribution of all the correlation coefficients that were calculated.","Sample with replacement from the X values.","Choose a new threshold for rejecting H0 (alpha)","Compare the mean of the X scores with the mean of the Y scores","Calculate the correlation (Pearson or Spearman).","Repeat the previous steps 10,000 times."]
  answers[block_id] = (step_1c, step_2c, step_3c, step_4c, step_5c, step_6c, step_7c)
  print("Your answer:", answers[block_id])
  executed_blocks.add(block_id)
except Exception as e:
  print(str(e))

Block 20#

In [ ]:
#@title **<font size="6" color='cornflowerblue'><ins>Question 7.6</ins>** 
#@markdown #**How would you visualize it to estimate H0 distribution?**
#@markdown - **Note** - A good graph here doesn't 100% guarentee that you were right in the previous question

block_id = 20
required_blocks = [13, 18]

def prepare_block():
  try:
    initialized
  except:
    raise Exception("You need to identify yourself in Block 1#")
  verify_dependencies(block_id, required_blocks)


def block20():
  x_axis = "Choose an answer" #@param ["Choose an answer","observation number","x values from the graph above", "calculated correlations"]
  y_axis = "Choose an answer" #@param ["Choose an answer","probability density", "y values from the graph above", "calculated correlations"]


  #plot their steps
  B=10000
  steps = [step_dict[step_1c],step_dict[step_2c],step_dict[step_3c],step_dict[step_4c]]#by their choice

  if (x_axis=="x values from the graph above") and (y_axis=="probability density"):
    plt.hist(x5)
  if (x_axis=="x values from the graph above") and (y_axis=="calculated correlations"):
    print("Cannot create this graph! choose x and y with the same number of observations")
  if (x_axis=="x values from the graph above") and (y_axis=="y values from the graph above"):
    plt.figure(figsize=(8,6));
    sns.regplot(x=x5, y=y5 ,fit_reg=False, color = "purple")
    plt.title(f"N={n5}, pearson correlation={cor_75}, p={pval_75}");


  if (x_axis == "calculated correlations"):
    if (y_axis == "calculated correlations"):
      calculated=step_dict[step_3c](step_dict[step_1c],step_dict[step_2c],x,y)
      print(9)
      plt.plot(calculated,calculated)
    elif (y_axis == "y values from the graph above"):
      print("Cannot create this graph! choose x and y with the same number of observations")
    elif (y_axis == "probability density"):
      try:
        steps[3](step_dict[step_1c],step_dict[step_2c],step_dict[step_3c],x5,y5) 
      except:
        if step_4c ==  "Create a distribution of all the correlation coefficients that were calculated.":
          if step_3c == "Repeat the previous two steps 10,000 times.":
            plt.figure(figsize=(7,7)) 
            plt.hist(np.random.normal(loc=.7,scale=.1,size=10000),bins=30, color="darkgreen")
          else:
            plt.figure(figsize=(7,7)) 
            plt.hist(np.repeat(cor_75,10000),bins=30, color="purple")
        if step_4c !=  "Create a distribution of all the correlation coefficients that were calculated.":
          print("You didn't plot anything.")

  if x_axis == "observation number":
    if y_axis=="calculated correlations":
          print("Cannot create this graph! choose x and y with the same number of observations")
    elif y_axis == "probability density":
      plt.hist(np.arange(n5))
    else:
      plt.plot(np.arange(n5),y5)

  plt.title("Your distribution");
  answers[block_id] = (x_axis, y_axis)
  print("Your answer:", answers[block_id])


try:
  prepare_block()
  eval('block'+str(block_id))()
  executed_blocks.add(block_id)
except Exception as e:
  print(str(e))

Block 21#

In [ ]:
#@title **<font size="6" color='cornflowerblue'><ins>Question 7.7</ins>** 
#@markdown #**Choose the steps to produce an estimation of the actual sampling distribution of the correlation and order them by the right order (not all of them must be used - fill "X" in excessive steps).**
#@markdown #**There is only one correct answer.**

block_id = 21
required_blocks = [13, 18]

def prepare_block():
  try:
    initialized
  except:
    raise Exception("You need to identify yourself in Block 1#")
  verify_dependencies(block_id, required_blocks)


try:
  prepare_block()
  step_1d = "X"#@param ["X","Mix the Y values between the different X values.","Sample with replacement from the pairs","Create a distribution of all the correlation coefficients that were calculated.","Sample with replacement from the X values.","Choose a new threshold for rejecting H0 (alpha)","Compare the mean of the X scores with the mean of the Y scores","Calculate the correlation (Pearson or Spearman).","Repeat the previous steps 10,000 times."]
  if step_1d == 'X':
    raise Exception("You must choose step_1.")
  step_2d = "X"#@param ["X","Mix the Y values between the different X values.","Sample with replacement from the pairs","Create a distribution of all the correlation coefficients that were calculated.","Sample with replacement from the X values.","Choose a new threshold for rejecting H0 (alpha)","Compare the mean of the X scores with the mean of the Y scores","Calculate the correlation (Pearson or Spearman).","Repeat the previous steps 10,000 times."]
  step_3d = "X"#@param ["X","Mix the Y values between the different X values.","Sample with replacement from the pairs","Create a distribution of all the correlation coefficients that were calculated.","Sample with replacement from the X values.","Choose a new threshold for rejecting H0 (alpha)","Compare the mean of the X scores with the mean of the Y scores","Calculate the correlation (Pearson or Spearman).","Repeat the previous steps 10,000 times."]
  step_4d = "X"#@param ["X","Mix the Y values between the different X values.","Sample with replacement from the pairs","Create a distribution of all the correlation coefficients that were calculated.","Sample with replacement from the X values.","Choose a new threshold for rejecting H0 (alpha)","Compare the mean of the X scores with the mean of the Y scores","Calculate the correlation (Pearson or Spearman).","Repeat the previous steps 10,000 times."]
  step_5d = "X"#@param ["X","Mix the Y values between the different X values.","Sample with replacement from the pairs","Create a distribution of all the correlation coefficients that were calculated.","Sample with replacement from the X values.","Choose a new threshold for rejecting H0 (alpha)","Compare the mean of the X scores with the mean of the Y scores","Calculate the correlation (Pearson or Spearman).","Repeat the previous steps 10,000 times."]
  step_6d = "X"#@param ["X","Mix the Y values between the different X values.","Sample with replacement from the pairs","Create a distribution of all the correlation coefficients that were calculated.","Sample with replacement from the X values.","Choose a new threshold for rejecting H0 (alpha)","Compare the mean of the X scores with the mean of the Y scores","Calculate the correlation (Pearson or Spearman).","Repeat the previous steps 10,000 times."]
  step_7d = "X"#@param ["X","Mix the Y values between the different X values.","Sample with replacement from the pairs","Create a distribution of all the correlation coefficients that were calculated.","Sample with replacement from the X values.","Choose a new threshold for rejecting H0 (alpha)","Compare the mean of the X scores with the mean of the Y scores","Calculate the correlation (Pearson or Spearman).","Repeat the previous steps 10,000 times."]
  answers[block_id] = (step_1d, step_2d, step_3d, step_4d, step_5d, step_6d, step_7d)
  print("Your answer:", answers[block_id])
  executed_blocks.add(block_id)
except Exception as e:
  print(str(e))

Block 22#

In [ ]:
#@title **<font size="6" color='cornflowerblue'><ins>Question 7.8</ins>** 
#@markdown #**How would you visualize this estimation of the sampling distribution of the correlation?**
#@markdown **Note** - A good graph here doesn't 100% guarentee that you were right in the previous question.

block_id = 22
required_blocks = [13, 18]

def prepare_block():
  try:
    initialized
  except:
    raise Exception("You need to identify yourself in Block 1#")
  verify_dependencies(block_id, required_blocks)


def block22():
  x_axis = "Choose an answer" #@param ["Choose an answer","observation number","x values from the graph above", "calculated correlations"]
  y_axis = "Choose an answer" #@param ["Choose an answer","probability density", "y values from the graph above", "calculated correlations"]


  #plot their steps
  B=10000


  steps = [step_dict[step_1d],step_dict[step_2d],step_dict[step_3d],step_dict[step_4d]]#by their choice

  if (x_axis=="x values from the graph above") and (y_axis=="probability density"):
    plt.hist(x5)
  if (x_axis=="x values from the graph above") and (y_axis=="calculated correlations"):
    print("Cannot create this graph! choose x and y with the same number of observations")
  if (x_axis=="x values from the graph above") and (y_axis=="y values from the graph above"):
    plt.figure(figsize=(8,6));
    sns.regplot(x=x5, y=y5 ,fit_reg=False, color = "purple")
    plt.title(f"N={n5}, pearson correlation={cor_75}, p={pval_75}");


  if (x_axis == "calculated correlations"):
    if (y_axis == "calculated correlations"):
      calculated=step_dict[step_3d](step_dict[step_1d],step_dict[step_2d],x,y)
      print(9)
      plt.plot(calculated,calculated)
    elif (y_axis == "y values from the graph above"):
      print("Cannot create this graph! choose x and y with the same number of observations")
    elif (y_axis == "probability density"):
      try:
        steps[3](step_dict[step_1d],step_dict[step_2d],step_dict[step_3d],x5,y5) 
      except:
        if step_4d ==  "Create a distribution of all the correlation coefficients that were calculated.":
          if step_3d == "Repeat the previous two steps 10,000 times.":
            plt.figure(figsize=(7,7)) 
            plt.hist(np.random.normal(loc=.7,scale=.1,size=10000),bins=30, color="darkgreen")
          else:
            plt.figure(figsize=(7,7)) 
            plt.hist(np.repeat(cor_75,10000),bins=30, color="purple")
        if step_4d !=  "Create a distribution of all the correlation coefficients that were calculated.":
          print("You didn't plot anything.")

  if x_axis == "observation number":
    if y_axis=="calculated correlations":
          print("Cannot create this graph! choose x and y with the same number of observations")
    elif y_axis == "probability density":
      plt.hist(np.arange(n5))
    else:
      plt.plot(np.arange(n5),y5)

  plt.title("Your distribution");

  answers[block_id] = (x_axis, y_axis)
  print("Your answer:", answers[block_id])


try:
  prepare_block()
  eval('block'+str(block_id))()
  executed_blocks.add(block_id)
except Exception as e:
  print(str(e))

Block 23#

In [ ]:
#@title **<font size="6" color='cornflowerblue'><ins>Question 7.9</ins>** 
#@markdown #**Look at the different graphs you got. What might be the difference between the data in Question 7.1 to the data in 7.5?**

block_id = 23
required_blocks = [13, 18]

def prepare_block():
  try:
    initialized
  except:
    raise Exception("You need to identify yourself in Block 1#")
  verify_dependencies(block_id, required_blocks)


def block23():
  Answer = 'Choose the correct answer' #@param ["Choose the correct answer", "Different sample size", "The world is unpredictable", "The means of the variables are different", "There is an outlier in the data"]
  if Answer == 'Choose the correct answer':
    raise Exception("You must choose an answer")
  answers[block_id] = Answer
  print("Your Answer is: ", answers[block_id])


try:
  prepare_block()
  eval('block'+str(block_id))()
  executed_blocks.add(block_id)
except Exception as e:
  print(str(e))


Block 24#

In [ ]:
#@title **<font size="6" color='cornflowerblue'><ins>Question 7.10</ins>** 
#@markdown #**How could you know the answer to the previous question in advance?**

block_id = 24
required_blocks = [13, 18]

def prepare_block():
  try:
    initialized
  except:
    raise Exception("You need to identify yourself in Block 1#")
  verify_dependencies(block_id, required_blocks)


def block24():
  Answer = 'Choose the correct answer' #@param ["Choose the correct answer", "Conduct a t-test", "Produce a graph of the data", "Examine the measures of central tendency of the distribution", "Correct the critical p-value (alpha)"]
  if Answer == 'Choose the correct answer':
    raise Exception("You must choose an answer")
  answers[block_id] = Answer
  print("Your Answer is: ", answers[block_id])


try:
  prepare_block()
  eval('block'+str(block_id))()
  executed_blocks.add(block_id)
except Exception as e:
  print(str(e))


**<p><font size="10"><font color='cornflowerblue'> Good job!</ins></p>** </font>

#**You finished answering the questions for this exercise! Here are some important things to consider:**


* You can view your answers by running the next block. 
*After changing an answer to a question, you will need to run the 'Show your answers' again for it to reflect changes.
* <b>If you change an answer to a question, you will need to run it again for changes to be saved. 


Block 25#

In [ ]:
#@title **<font color = cornflowerblue> <font size = 6>Show Your Answers <br>(Run to refresh after saving a question)**

block_id = 25


def prepare_block():
  try:
    initialized
  except:
    raise Exception("You need to identify yourself in Block 1#")
  verify_dependencies(block_id, range(block_id))


def answers_printer():
  for block in sorted(answers):
    print("Question " + str(block_mapping[block]) + ": " + str(answers[block]))

try:
  prepare_block()
  answers_printer()
  # blocks_run[block_id] = True
except Exception as e:
  print(str(e))


If you are ready to submit your answers, go ahead and press 'Save!'💾 .


This will prompt you to download your answers file. 


After downloading you will need to submit the file to moodle.
Good luck!



Block 26#

In [ ]:
#@title  **<p><font size="10"><font color='cornflowerblue'><ins> <---  💾 SAVE! </ins></p>**
#@markdown If you are ready to submit your answers, go ahead and press 'Save!'💾 . This will prompt you to download your answers file. After downloading you will need to submit the file to moodle. Good luck!

from google.colab import files
from shutil import make_archive
from os import path, remove
from zipfile import ZipFile

block_id = 26

def prepare_block():
  try:
    initialized
  except:
    raise Exception("You need to identify yourself in Block 1#")
  verify_dependencies(block_id, range(block_id- 1))


def answer_saver(file):
  for block in sorted(answers):
    file.write("Question " + str(block_mapping[block]) + ": " + str(answers[block])+'\n')


def submission_saver():
  # Writing answers
  f = open("ex"+str(exercise_number)+"_submission", "w")
  f.write("#"+str(id_number)+"\n")
  f.write("#"+str(group_number)+"\n")
  answer_saver(f) 
  f.write("#End of answers")
  f.close()

  # Writing data
  g = open("ex"+str(exercise_number)+"_data", "w")
  g.write(str(data))
  g.close()
  datazip = ZipFile("ex"+str(exercise_number)+"_data"+'.zip', 'w')
  datazip.write("ex"+str(exercise_number)+"_data")
  datazip.close()

  # Creating submission file
  zipObj = ZipFile("ex"+str(exercise_number)+'.zip', 'w')
  zipObj.write("ex"+str(exercise_number)+'_submission')
  zipObj.write("ex"+str(exercise_number)+"_data")
  zipObj.close()
  files.download("ex"+str(exercise_number)+'.zip') 

  # Message to user
  print("Well done! You are now downloading your submission file")


try:
  prepare_block()
  submission_saver()
except Exception as e:
  print(str(e))

